Функции активации  в нейронных сетях
====================

Согласно Википедии - В нейронных сетях, функция активации нейрона определяет выход нейрона из входного значения.
Чтобы разобраться, что и как проиходит внутри нейросети, а также как влияет функция активации на выходные значения, следует определить следующие понятия:
* **Входные значения/Inputs/x** - в самом начале, это те, данные на основе которых следует выполнить предсказание для выходного значений.
* **Входные значения/Inputs/y** - те значение, которые необходимо предсказать
* **Веса/weights/w** - каждому входному х соответствуют какие-то w, которые инициализируются перед обучением, и оптимизируются в процессе обучения. Количество весов для каждого нейрона, зависит от количества x.
* **Отступ/bias/b** - Аналогично весам, инициализируется до обучения и оптимизируется в процессе обучения, но в отличие от весов, bias - это одно число. 
* **Сумматорая функция $S$** - функция вида $xw+b$, где w - weights для входов x, b - bias.
* **Функция активации $A$** - функция, которая применяется к сумматорной, задается до обучения сети, на этапе построения модели.  

Для обычной нейронной сети, с прямым распространением(сигналов(которые на самом деле, какие-то числа)) с 1 нейроном в скрытом слое, последовательность "предсказывания"(вычисления) выходного значения такая:
1. Взять $x$, применить к нему сумматорную функцию, с весами и отступом для скрытого нейрона.
2. К полученному значению функцию активации.
3. Применить к значению из пункта 2 сумматорную функцию, с весами и отступом на выходном слое.
4. Финальный этап: применить функцию активации на выходном слое.  
  
Функция активации играет важнейшую роль, какую именно рассмотрим ниже.

In [93]:
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from ipywidgets import interactive, interact, FloatSlider, HBox, VBox, Dropdown
import ipywidgets as widgets

%matplotlib inline

Устанавлием параметры графика

In [104]:
rcParams['figure.figsize'] = 13, 6

def interact_hookup(f, controls):
    from ipywidgets import Output
    out = Output()
    def observer(change):
        out.clear_output()
        kwargs = {k:v.value for k,v in controls.items()}
        with out:
            f(**kwargs)
    for k,w in controls.items():
        w.observe(observer, 'value')
    observer(None)
    return out

## Какие бывают функции активации:

**Linear** - обычная линейная функция $y=kx+b$, где $k=w$ - weight(вес нейрона), $b = bias$ - отступ,
формула: $$A(X)=weights*X+bias$$
В какой-то степени, не является активационной функцией вовсе, так как она же и есть сумматорная функция. Исходя из этого, когда **говорят, что функция активации линейная** - это значит функция активации не применяется вовсе. Линейную функцию активации, применяют там, где зависимость между входными данными и целевой переменной линейная или данные линейно разделимы(для задачи классификации).

**Sigmoid(Logistic) a.k.a. Soft step** - функция вида: $$Sigmoid(x)=\frac{1}{1+e^{-x}} $$
Область значений для этой функции $(0;1)$. В сигмойде, для $x<\pm 6$ выходные значения будут: 
$$\begin{cases} 1, & \mbox{если } x > 0 \\ 0, & \mbox{если } x > 0 \end{cases}$$
В случае, если во входных данных $x>\pm 6$, то применяя сигмойду на этих значениях, теряется информативность, в виду того что, они либо обнуляются, либо равны 1. Во избежании этого, необходимо масштабировать входные значения, то есть привести к меньшему промежутку, например разделить на максимальный возможный x из обучающих данных.

**TanH** - [функция гиперболического тангенса](https://en.wikipedia.org/wiki/Hyperbolic_function#Hyperbolic_tangent), в отличии от сигмоиды, может принимать значения на отрезке $(-1;1)$. Для этой функции активации, веса должны быть инициализированы малыми случайными значениями. $$tanh(x) = \frac{2}{1+e^{-2x}} - 1$$
С тангесом, промежуток потери информации, ещё меньше, поэтому необходимо "пошаманить" над данными, также как и с сигмойдой.

**ReLU - Rectified Linear Unit** - дословный перевод: Выпрямленный линейный нейрон. 
$$ReLU(x)=max(0,x)$$
То есть, если $x \in (-\infty , 0)$, то $ReLU(x)=0$, иначе $ReLU(x)=x$  
Наиболее популярная функция активации, однако, имеет ряд недостатков: 
* Недифференцируема в нуле, но диффиринцируемая сколь угодно близко к нулю, но не в самом нуле.
* Нейроны с активацией ReLU могут умереть: случается так, что при определённых значениях весов, выходные значения сети обнуляются, то есть $ReLU(x)=0, \forall x \in Inputs$

#### Почему ReLU популярна, хотя имеет ряд недостатков?
Она хорошо себя показывает на практике, но при обучении, <font color="red">**не следует**</font> устанавливать большой learning rate.
Также, проблемы ReLU, решаются следующими изменениями:

**Elu** - Exponential Linear Unit - улучшенная функция ReLU, записывается она так:
$$ELU(x)= \begin{cases} x, & \mbox{если } x \geq 0 \\ \alpha*(e^x-1), & \mbox{если } x <0 \end{cases}$$
По-умолчанию, $\alpha=1$.

**Leaky ReLU** - функция активации представленная в 2014 году, авторы которой: *Andrew L. Maas, Awni Y. Hannun, Andrew Y. Ng(Очень популярный человек в машинном обучении)*, призванная устранить проблемы ReLU.
$${Leaky \space ReLU}(x)=\begin{cases} x, & \mbox{если } x>0 \\ 0.01x, & \mbox иначе \end{cases}$$

# Функции активации на практике:

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def tanh(x):
    return 2/(1+np.exp(-2*x)) - 1

def relu(x):
    return max(0,x)

def leaky_relu(x):
    return x if x > 0 else 0.01*x

def elu(x, alpha=1):
    return x if x > 0 else alpha*(np.exp(x)-1)

## Графики:

Можно выбрать функцию активации из выпадающего списка.

In [131]:
def f(activation=(sigmoid, 'Sigmoid')):
    x = np.linspace(-7,5,100)
    plt.plot(x, list(map(activation[0], x)), label='{0} activation'.format(activation[1]))
    plt.grid()
    plt.title('{0} function'.format(str(activation[1])))
    plt.xlabel('X')
    plt.ylabel('Output')
    plt.legend()
    plt.show()


interactive_plot = interactive(f,activation={'Sigmoid':(sigmoid, 'Sigmoid'), 'TanH':(tanh,'TanH'),
                                             'ReLU':(relu,'ReLU'), 'ELU':(elu,'ELU'), 
                                             'Leaky ReLU':(leaky_relu,'Leaky ReLU')})
output = interactive_plot.children[-1]
display(interactive_plot)

A Jupyter Widget

Здесь представлен выход Feed-Forward Neural Network с 1 нейроном, то есть:
$$A(S)$$
Ползунки позволяют менять значение w(вес нейрона) и b(bias, отступ нейрона), и посмотреть как изменяется выход сети.

In [5]:
def f(activation=(sigmoid, 'Sigmoid'), w=2, b=1):
    plt.figure(2)
    x = np.linspace(-5, 5, num=1000)
    plt.plot(x, list(map(activation[0], x*w+b)), label='w={0}, b={1}'.format(w,b))
    plt.grid()
    
    plt.title('{0} neuron'.format(str(activation[1])))
    plt.xlabel('X')
    plt.ylabel('output')
    plt.legend()
    plt.show()

interactive_plot = interactive(f,activation={'Sigmoid':(sigmoid, 'Sigmoid'), 'TanH':(tanh,'TanH'),
                                             'ReLU':(relu,'ReLU'), 'ELU':(elu,'ELU'), 
                                             'Leaky ReLU':(leaky_relu,'Leaky ReLU')},
                                w=(-50.0, 50.0), b=(-10, 10))
output = interactive_plot.children[-1]
interactive_plot

A Jupyter Widget

График аналогичный предыдущему, только теперь в НН 1 скрытый слой с 2-мя нейронами, на выходе требуется 1 значение.

<center> **Таблица для аппроскимации фукнции $y=x^2$** </center>  

|       | Sigmoid | TanH  |  ReLU |  ELU  | Leaky ReLU |
|:-----:|---------|-------|-------|-------|:----------:|
| $w_1$ |  1.30   | 0.66  |  1.40 | 1.30  |    0.80    |
| $b_1$ | -2.50   | -0.90 | -0.90 | -1.90 |    -0.40   |
| $w_2$ | -1.30   | -0.66 | -1.40 | -1.30 |    -0.80   |
| $b_2$ | -2.50   | -0.90 | -0.90 | -1.90 |    -0.40   |
| $w_3$ |  10.00  | 5.00  |  1.80 | 3.00  |    3.50    |
| $b_3$ | -1.50   | 7.00  |  0.00 | 5.00  |    0.00    |
| $w_4$ |  10.00  | 5.00  |  1.80 | 3.00  |    3.50    |

In [130]:
def f(w1, b1, w2, b2, w3, b3, w4, activation=(sigmoid, 'Sigmoid')):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    A1 = np.array(list(map(activation[0], x*w1+b1)))
    A2 = np.array(list(map(activation[0], x*w2+b2)))
    A3 = A1*w3 + A2*w4 + b3
    plt.plot(x, A1, linestyle='--', label='First neuron activation, w={0}, b={1}'.format(w1,b1))
    plt.plot(x, A2, linestyle='--', label='Second neuron activation, w={0}, b={1}'.format(w2,b2))
    
    plt.plot(x, A1*w3 + b3, linestyle='--', label='First neuron output, w={0}, b={1}'.format(w3,b3))
    plt.plot(x, A2*w4 + b3, linestyle='--', label='Second neuron output, w={0}, b={1}'.format(w4,b3))
    
    plt.plot(x, A3, color='r', linewidth=5.0, 
             label='Output value')
    plt.plot(x, x**2, linewidth=3.0, linestyle=':', label=r'$y=x^2$ function')
    plt.grid()
    plt.ylim(-1, 5)
    plt.title('1 hidden layer, with 2 {0} neurons'.format(str(activation[1])))
    plt.xlabel('X')
    plt.ylabel('Output')
    plt.legend()
    plt.show()

params = dict(activation = Dropdown(options={'Sigmoid':(sigmoid, 'Sigmoid'), 'TanH':(tanh,'TanH'),
                                             'ReLU':(relu,'ReLU'), 'ELU':(elu,'ELU'), 
                                             'Leaky ReLU':(leaky_relu,'Leaky ReLU')}, 
                                              description='Activation: '),
                           w1=FloatSlider(value= 1.3, min=-5, max=11, description='w1:'),
                           w2=FloatSlider(value=-1.3, min=-5, max=11, description='w2:'),
                           w3=FloatSlider(value=  10, min=-10,max=11, description='w3:'),
                           w4=FloatSlider(value=  10, min=-10,max=11, description='w4:'),
                           b1=FloatSlider(value=-2.5, min=-5, max=11, description='b1:'),
                           b2=FloatSlider(value=-2.5, min=-5, max=11, description='b2:'),
                           b3=FloatSlider(value=-1.5, min=-5, max=11, description='b3:'))  

output = interact_hookup(f, params)
ok = VBox([params['activation'],
           HBox([params['w1'], params['w2']]),  
           HBox([params['w3'], params['w4']]),
           HBox([params['b1'], params['b2']]),
           params['b3'],output])
display(ok)

A Jupyter Widget

Из примера становиться ясно:
* 2 сигмоидных нейрона, в точности повторяют $x^2$
* ELU засчет изгиба для $x<0$, очень хорошо справляются с задачей
* TanH требует тонкой настройки параметров
* ReLU и Leaky ReLU справляются с задачей хуже из-за отсутствия нелинейности

## 1 скрытый слой, с 3-мя нейронами

In [17]:
def f(activation=(sigmoid, 'Sigmoid'),
                  w1=-1, w2=2, w3=0.4, w4=1,  
                  w5=1, w6=1,
                  b1=0, b2=0, b3=2, b4=0):
    
    plt.figure(2)
    
    x = np.linspace(-10, 10, num=1000)
    
    A1 = np.array(list(map(activation[0], x*w1+b1)))
    A2 = np.array(list(map(activation[0], x*w2+b2)))
    A3 = np.array(list(map(activation[0], x*w3+b3)))
    
    A4 = A1*w4 + A2*w5 + A3*w6 + b4
    
    plt.plot(x, A1, linestyle='--', label='1st neuron activation, w={0}, b={1}'.format(w1,b1))
    plt.plot(x, A2, linestyle='--', label='2nd neuron activation, w={0}, b={1}'.format(w2,b2))
    plt.plot(x, A3, linestyle='--', label='3rd neuron activation, w={0}, b={1}'.format(w3,b3))
    plt.plot(x, A4, color='r', linewidth=5.0, 
             label='Output value, w3={0}, b3={1}, w4={0}, b4={1}'.format(w3,b3,w4,b4))
    plt.plot(x, x**2, linewidth=3.0, linestyle=':', label='Sine function')
    plt.grid()
    plt.ylim(-1, 5)
    plt.title('1 hidden layer, with 3 {0} neurons'.format(str(activation[1])))
    plt.xlabel('X')
    plt.ylabel('Output')
    plt.legend()
    plt.show()

interactive_plot = interactive(f,activation={'Sigmoid':(sigmoid, 'Sigmoid'), 'TanH':(tanh,'TanH'),
                                             'ReLU':(relu,'ReLU'), 'ELU':(elu,'ELU'), 
                                             'Leaky ReLU':(leaky_relu,'Leaky ReLU')},  
                                w1=(-5.0, 5.0), w2=(-5.0, 5.0), w3=(-5.0, 5.0), w4=(-5.0, 5.0),
                                w5=(-5.0, 5.0), w6=(-5.0, 5.0), b1=(-5.0, 5.0), b2=(-5.0, 5.0),
                                b3=(-5.0, 5.0), b4=(-5.0, 5.0))
output = interactive_plot.children[-1]
interactive_plot

A Jupyter Widget

## 2 скрытых слоя, по 3 нейрона в каждом 

### $W{_i}{^j}$ - $i$ - слой, $j$ - индекс нейрона

In [122]:
def f(activation=(sigmoid, 'Sigmoid'),
      w11=0.5, w12=1, w13=1,
      w21=0.5, w22=1, w23=1, w24=1, w25=1, w26=1, w27=1, w28=1, w29=1,
      w31=0.5, w32=1, w33=1, 
      b11=0.5, b12=0, b13=0, b21=0, b22=0, b23=0, b3=0):
    
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    
    A11 = np.array(list(map(activation[0], x*w11+b11)))
    A12 = np.array(list(map(activation[0], x*w12+b12)))
    A13 = np.array(list(map(activation[0], x*w13+b13)))
    
    A21 = np.array(list(map(activation[0], A11*w21 + A12*w22 + A13*w23 + b21)))
    A22 = np.array(list(map(activation[0], A11*w24 + A12*w25 + A13*w26 + b22)))
    A23 = np.array(list(map(activation[0], A11*w27 + A12*w28 + A13*w29 + b23)))
    
    A3 = w31*A21+w32*A22+w33*A23+b3
    
    plt.plot(x, A11, linestyle='--', label='1st layer 1 neuron activation')
    plt.plot(x, A12, linestyle='--', label='1st layer 2 neuron activation')
    plt.plot(x, A13, linestyle='--', label='1st layer 3 neuron activation')
    
    plt.plot(x, A21, linestyle='--', label='2nd layer 1 neuron activation')
    plt.plot(x, A22, linestyle='--', label='2nd layer 2 neuron activation')
    plt.plot(x, A23, linestyle='--', label='2nd layer 3 neuron activation')
    
    plt.plot(x, A3, color='r', linewidth=5.0, label='Model output value')
    
    plt.plot(x, x**2, linewidth=3.0, linestyle=':', label='sqr(x) function')
    plt.grid()
    plt.ylim(-1, 5)
    plt.title('Model with 2 hidden layer, each have 3 {0} neurons'.format(activation[1]))
    plt.xlabel('X')
    plt.ylabel('output')
    plt.legend()
    plt.show()

interactive_plot = interactive(f,activation={'Sigmoid':(sigmoid, 'Sigmoid'), 'TanH':(tanh,'TanH'),
                                             'ReLU':(relu,'ReLU'), 'ELU':(elu,'ELU'), 
                                             'Leaky ReLU':(leaky_relu,'Leaky ReLU')}, 
                               w11=(-10.0, 10.0),w12=(-10.0, 10.0),w13=(-10.0, 10.0),
                               w21=(-10.0, 10.0),w22=(-10.0, 10.0),w23=(-10.0, 10.0),
                               w24=(-10.0, 10.0),w25=(-10.0, 10.0),w26=(-10.0, 10.0),
                               w27=(-10.0, 10.0),w28=(-10.0, 10.0),w29=(-10.0, 10.0),
                               w31=(-10.0, 10.0),w32=(-10.0, 10.0),w33=(-10.0, 10.0),
                               b11=(-10.0, 10.0),b12=(-10.0, 10.0),b13=(-10.0, 10.0),
                               b21=(-10.0, 10.0),b22=(-10.0, 10.0),b23=(-10.0, 10.0),
                               b3= (-10.0, 10.0),)
output = interactive_plot.children[-1]
interactive_plot

A Jupyter Widget

# Заключение:  
Нейронная сеть, может всё. Но следует тщательно подбирать активационную функцию. Также необходимо проводить предобработку данных, перед обучением.


<div style="text-align: right">**made by <font color="orange">[@iamlion12](https://t.me/iamlion12)</font>**</div>